## PyPDF
- https://pypi.org/project/pypdf/
- https://pypdf.readthedocs.io/en/stable/

In [2]:
# pip install pypdf

In [34]:
import pypdf
import spacy
spacy.load("es_core_news_sm")

import es_core_news_md
nlp = es_core_news_md.load()

In [50]:
import re

In [12]:
# creating a pdf reader object
reader = pypdf.PdfReader('HistoriaDelTiempo.pdf')

# print the number of pages in pdf file
print(len(reader.pages))

# print the text of the first page
# print(reader.pages[0].extract_text())

177


#### Lectura

In [36]:
texto = reader.pages[105].extract_text()
doc = nlp(texto)
tokens = [token.text for token in doc]

#### Detección de Oraciones

In [41]:
oraciones= list(doc.sents)
print("Cantidad de oraciones:")
print(len(oraciones))

print("\nOraciones:\n\n")
for x in oraciones:
    print(x)
    print("\n")

Cantidad de oraciones:
7

Oraciones:


Historia del Tiempo: Del Big Bang a los Agujeros Negros                                                    Stephen Hawking 
 106
gramo.  


A pesar de ello, fallarán al final de la vida del agujero negro cuando su masa 
se haga muy pequeña.  


El resultado más probable p arece que será que el agujero 
negro simplemente desaparecerá, al menos de nuestra región del universo, 
llevándose con él al astronauta y a cualquier singularidad que pudiera contener, si en 
verdad hay alguna.  


Esto fue la primera indicación de que la mecánica cuántica 
podría eliminar las singularidades predichas por la teoría de la relatividad.  


Sin 
embargo, los métodos que otros científicos y yo utilizábamos en 1974 no eran 
capaces de responder a cuestiones como la de si debían existir singularidades en la 
gravedad cuántica; A partir de 1975, comencé a desarrollar una aproximación más 
potente a la gravedad cuántica basada en la idea de Feynman de suma sobre las 
hi

#### Tokenización

In [42]:
for token in doc:
  print(token, token.idx)

Historia 0
del 9
Tiempo 13
: 19
Del 21
Big 25
Bang 29
a 34
los 36
Agujeros 40
Negros 49
                                                    56
Stephen 107
Hawking 115

  123
106 125

 128
gramo 129
. 134
  136
A 137
pesar 139
de 145
ello 148
, 152
fallarán 154
al 163
final 166
de 172
la 175
vida 178
del 183
agujero 187
negro 195
cuando 201
su 208
masa 211

 216
se 217
haga 220
muy 225
pequeña 229
. 236
  238
El 239
resultado 242
más 252
probable 256
p 265
arece 267
que 273
será 277
que 282
el 286
agujero 289

 297
negro 298
simplemente 304
desaparecerá 316
, 328
al 330
menos 333
de 339
nuestra 342
región 350
del 357
universo 361
, 369

 371
llevándose 372
con 383
él 387
al 390
astronauta 393
y 404
a 406
cualquier 408
singularidad 418
que 431
pudiera 435
contener 443
, 451
si 453
en 456

 459
verdad 460
hay 467
alguna 471
. 477
  479
Esto 480
fue 485
la 489
primera 492
indicación 500
de 511
que 514
la 518
mecánica 521
cuántica 530

 539
podría 540
eliminar 547
las 556
singularidades 560

In [43]:
for token in doc:
  print(token, token.idx, token.text_with_ws,
        token.is_alpha, token.is_punct, token.is_space,
        token.shape_, token.is_stop)

Historia 0 Historia  True False False Xxxxx False
del 9 del  True False False xxx True
Tiempo 13 Tiempo True False False Xxxxx False
: 19 :  False True False : False
Del 21 Del  True False False Xxx True
Big 25 Big  True False False Xxx False
Bang 29 Bang  True False False Xxxx False
a 34 a  True False False x True
los 36 los  True False False xxx True
Agujeros 40 Agujeros  True False False Xxxxx False
Negros 49 Negros  True False False Xxxxx False
                                                    56                                                     False False True      False
Stephen 107 Stephen  True False False Xxxxx False
Hawking 115 Hawking  True False False Xxxxx False

  123 
  False False True 
  False
106 125 106 False False False ddd False

 128 
 False False True 
 False
gramo 129 gramo True False False xxxx False
. 134 .  False True False . False
  136   False False True   False
A 137 A  True False False X True
pesar 139 pesar  True False False xxxx True
de 145 de  True

#### Stopwords

In [45]:
# Saco todas las stopwords del idioma español

spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
print(len(spacy_stopwords))
for stop_word in list(spacy_stopwords)[:10]:
  print(stop_word)

521
consigue
tanto
hablan
última
en
algunos
cada
por
todas
hacia


In [46]:
for token in doc:
  if not token.is_stop:
    print(token)

Historia
Tiempo
:
Big
Bang
Agujeros
Negros
                                                   
Stephen
Hawking

 
106


gramo
.
 
,
fallarán
vida
agujero
negro
masa


haga
pequeña
.
 
resultado
probable
p
arece
agujero


negro
simplemente
desaparecerá
,
región
universo
,


llevándose
astronauta
singularidad
pudiera
contener
,


.
 
indicación
mecánica
cuántica


eliminar
singularidades
predichas
teoría
relatividad
.
 


,
métodos
científicos
utilizábamos
1974


capaces
responder
cuestiones
debían
existir
singularidades


gravedad
cuántica
;
1975
,
comencé
desarrollar
aproximación


potente
gravedad
cuántica
basada
idea
Feynman
suma


historias
posibles
.
 
respuestas
aproximación
sugiere
origen


destino
universo
contenidos
,
tales
astronautas
,
descritas


capítulos
siguientes
.
 
verá
,
principio
incertidumbre


establece
limitaciones
precisión
predicciones
,


tiempo
eliminar
incapacidad
predicción
carácter
fundamental
ocurre


singularidad
espacio-tiempo
.

 


In [47]:
documento_sin_stopword = [token for token in doc if not token.is_stop]
print(documento_sin_stopword)

[Historia, Tiempo, :, Big, Bang, Agujeros, Negros,                                                    , Stephen, Hawking, 
 , 106, 
, gramo, .,  , ,, fallarán, vida, agujero, negro, masa, 
, haga, pequeña, .,  , resultado, probable, p, arece, agujero, 
, negro, simplemente, desaparecerá, ,, región, universo, ,, 
, llevándose, astronauta, singularidad, pudiera, contener, ,, 
, .,  , indicación, mecánica, cuántica, 
, eliminar, singularidades, predichas, teoría, relatividad, .,  , 
, ,, métodos, científicos, utilizábamos, 1974, 
, capaces, responder, cuestiones, debían, existir, singularidades, 
, gravedad, cuántica, ;, 1975, ,, comencé, desarrollar, aproximación, 
, potente, gravedad, cuántica, basada, idea, Feynman, suma, 
, historias, posibles, .,  , respuestas, aproximación, sugiere, origen, 
, destino, universo, contenidos, ,, tales, astronautas, ,, descritas, 
, capítulos, siguientes, .,  , verá, ,, principio, incertidumbre, 
, establece, limitaciones, precisión, predicciones, ,, 


In [48]:
for token in doc:
  print(token, '-', token.lemma_)

Historia - Historia
del - del
Tiempo - Tiempo
: - :
Del - Del
Big - Big
Bang - Bang
a - a
los - el
Agujeros - Agujeros
Negros - Negros
                                                    -                                                    
Stephen - Stephen
Hawking - Hawking

  - 
 
106 - 106

 - 

gramo - gramo
. - .
  -  
A - a
pesar - pesar
de - de
ello - él
, - ,
fallarán - fallar
al - al
final - final
de - de
la - el
vida - vida
del - del
agujero - agujero
negro - negro
cuando - cuando
su - su
masa - masa

 - 

se - él
haga - hacer
muy - mucho
pequeña - pequeño
. - .
  -  
El - el
resultado - resultado
más - más
probable - probable
p - p
arece - arecir
que - que
será - ser
que - que
el - el
agujero - agujero

 - 

negro - negro
simplemente - simplemente
desaparecerá - desaparecer
, - ,
al - al
menos - menos
de - de
nuestra - nuestro
región - región
del - del
universo - universo
, - ,

 - 

llevándose - llevar él
con - con
él - él
al - al
astronauta - astronauta
y - y
a - a
cualqu

#### Word Frequency

In [51]:
texto1 = re.sub('\n', '', texto) #remover saltos de linea
print(type(texto))
str(texto1)

<class 'str'>


'Historia del Tiempo: Del Big Bang a los Agujeros Negros                                                    Stephen Hawking  106gramo.  A pesar de ello, fallarán al final de la vida del agujero negro cuando su masa se haga muy pequeña.  El resultado más probable p arece que será que el agujero negro simplemente desaparecerá, al menos de nuestra región del universo, llevándose con él al astronauta y a cualquier singularidad que pudiera contener, si en verdad hay alguna.  Esto fue la primera indicación de que la mecánica cuántica podría eliminar las singularidades predichas por la teoría de la relatividad.  Sin embargo, los métodos que otros científicos y yo utilizábamos en 1974 no eran capaces de responder a cuestiones como la de si debían existir singularidades en la gravedad cuántica; A partir de 1975, comencé a desarrollar una aproximación más potente a la gravedad cuántica basada en la idea de Feynman de suma sobre las historias posibles.  Las respuestas que esta aproximación sugier

In [53]:
docWordFreq = nlp(texto1)

# Remover stopwords
words= [token.text for token in doc if not token.is_stop and not token.is_punct]
from collections import Counter
word_freq= Counter(words)

# Sacar las 5 mas frecuentes y sus frecuencias
common_words= word_freq.most_common(5)
print(common_words)

[('\n', 16), (' ', 6), ('cuántica', 3), ('\n ', 2), ('agujero', 2)]


In [54]:
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print(unique_words)

['Historia', 'Tiempo', 'Big', 'Bang', 'Agujeros', 'Negros', '                                                   ', 'Stephen', 'Hawking', '106', 'gramo', 'fallarán', 'vida', 'masa', 'haga', 'pequeña', 'resultado', 'probable', 'p', 'arece', 'simplemente', 'desaparecerá', 'región', 'llevándose', 'astronauta', 'pudiera', 'contener', 'indicación', 'mecánica', 'predichas', 'teoría', 'relatividad', 'métodos', 'científicos', 'utilizábamos', '1974', 'capaces', 'responder', 'cuestiones', 'debían', 'existir', '1975', 'comencé', 'desarrollar', 'potente', 'basada', 'idea', 'Feynman', 'suma', 'historias', 'posibles', 'respuestas', 'sugiere', 'origen', 'destino', 'contenidos', 'tales', 'astronautas', 'descritas', 'capítulos', 'siguientes', 'verá', 'principio', 'incertidumbre', 'establece', 'limitaciones', 'precisión', 'predicciones', 'tiempo', 'incapacidad', 'predicción', 'carácter', 'fundamental', 'ocurre', 'espacio-tiempo']


#### Lematización

In [1]:
for token in doc:
  print(token, '-', token.lemma_)

NameError: name 'texto1' is not defined